In [21]:
import numpy as np
from csv import DictReader as dr
import pandas as pd


In [22]:
data=pd.read_csv('/heart_data.csv')

In [23]:
def confusion_mat(y_test,y_pred):
    lamb=np.zeros([2, 2], dtype = int)
    for i in range(len(y_test)):
        if(y_test[i]==1 and y_pred[i]==1):
            lamb[0][0]+=1
        elif(y_test[i]==1 and y_pred[i]==0):
            lamb[1][0]+=1
        elif(y_test[i]==0 and y_pred[i]==0):
            lamb[1][1]+=1
        else:
            lamb[0][1]+=1
    return lamb


In [24]:
def sigmoid_func(x):
    return (1/(1+np.exp(-x)))

def computeError(predicted, actual):
    err = np.mean(predicted != actual)
    return err * 100

In [25]:
def find_y(Y):
    m=len(Y)
    for i in range(m):
       # print(Y[i])
        if Y[i]>=0.5:
            Y[i]=1
        else:
            Y[i]=0
    return Y



In [26]:
def denormalise_price(price):
    global mean
    global stddev
    #print(price)
    ret = price * stddev[13] + mean[13]
    #print()
    #print(ret)
    return ret




In [27]:
def batch_gradient(X_train,Y_train):
    interations=1000
    alpha = 0.01        
    W0 = 0
    W1 = 0
    sz = len(X_train)
            
    costs = []
    W = [[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
    W=np.array(W)
    lamda=0.01 *(np.ones(W.shape))
    #print(X_train.shape)
    for epoc in range(interations):
        s=sigmoid_func(X_train.dot(W))
       # print(s.shape,Y_train.shape)
        d=(s-Y_train)
       # print(d.shape)
        #print(d.shape)
        diff=X_train.T.dot(d)*(alpha/sz)
       # print(diff.shape)
        W = W*(1-alpha*lamda/sz) - diff
        new_Cost = np.sum((sigmoid_func(X_train.dot(W))-Y_train)**2)/(2*sz)
        costs.append(new_Cost)
    return (costs,W)

In [28]:

mean = data.mean()
stddev = data.std()

data = (data - data.mean())/data.std()

X=data.iloc[:,:-1]
Y=data.iloc[:,-1:]

X = np.array(X)
Y = np.array(Y)

one = np.ones((len(X),1))
X = np.concatenate((one,X),axis=1)
totalsize = len(X)
trainsize = int(totalsize*0.7)

X_train = X[:trainsize]
X_test = X[trainsize:]
Y_train = Y[:trainsize]
Y_test = Y[trainsize:]
costs,W=batch_gradient(X_train,Y_train)


In [31]:
print("Batch_Gradient Feature Scaling ")
# print("Parameters:\n",W)
print()

Y_pred=sigmoid_func(np.dot(X_test,W))
Y_pred=denormalise_price(Y_pred)

Y_pred=find_y(Y_pred)

er=computeError(Y_pred,denormalise_price(Y_test))
print("Error:",er,"%\nAccuracy:",100-er,"%")
print("----------------------------\n")
conf=confusion_mat(denormalise_price(Y_test),Y_pred)
acc=(conf[0][0]+conf[1][1])/(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1])

print("According to Confusion matrix:")
print("")
print("Accuracy:",acc*100)

recall=conf[0][0]/(conf[1][0]+conf[0][0])

print("Recall:",recall*100)
pre=conf[0][0]/(conf[0][1]+conf[0][0])

print("Precision:",pre*100)
tnr=conf[1][1]/(conf[0][1]+conf[1][1])

print("True Negative rate:",tnr*100)

Batch_Gradient Feature Scaling 

Error: 19.78021978021978 %
Accuracy: 80.21978021978022 %
----------------------------

According to Confusion matrix:

Accuracy: 80.21978021978022
Recall: 69.76744186046511
Precision: 85.71428571428571
True Negative rate: 89.58333333333334

